In [6]:
import pandas as pd
import numpy as np
import json

In [7]:
filepath = "../data/HS_postings.json"

# Opening JSON file
f = open(filepath)
  
# Returns JSON object as a dictionary
data = json.load(f)

In [8]:
print(data['postings'][0].keys())

dict_keys(['type', 'added_institution_label_ids', 'apply_start', 'college_ids', 'created_at', 'updated_at', 'id', 'institution_label_ids', 'interview_status', 'employer_logo', 'employer_name', 'employer_id', 'employment_type_name', 'expiration_date', 'job_id', 'job_name', 'major_group_ids', 'canonical_major_group_ids', 'major_ids', 'major_names', 'school_id', 'top_employer', 'posting_quality', 'job', 'job_reviews_count', 'has_relationship_opportunity', 'first_availability_calendar_id'])


In [9]:
print(data['postings'][0]['job'].keys())

dict_keys(['ats_integrated', 'ats_provider', 'division_id', 'division_name', 'duration', 'employer_id', 'employer_industry_id', 'employer_industry_name', 'employer_group_ids', 'employer_logo_url', 'employer_name', 'employer_path', 'employment_type_id', 'employment_type_name', 'id', 'job_type_id', 'job_type_name', 'location_ids', 'location_cities', 'location_countries', 'location_names', 'location_points', 'location_states', 'locations_option', 'salary_type_id', 'salary_type_name', 'text_description', 'title', 'type', 'work_study', 'canonical_major_group_ids', 'canonical_major_group_count', 'remote', 'job_role_group_ids', 'student_screen', 'messageable_job_user_profile', 'scheduleable_job_user_profile', 'job_apply_setting', 'required_job_document_types', 'student_qualifications', 'job_user_profiles', 'employer_calendar_availability_enabled', 'virtual_fair_opportunity', 'relationship_opportunities', 'student_employee_profiles'])


## Make Companies CSV

In [10]:
cols = ['id', 'apply_start', 'created_at', 'updated_at',  
        'employer_logo', 'employer_name', #'interview_status', 'school_id', 'top_employer', 'job_reviews_count'
        'employer_id', 'employment_type_name', 'expiration_date', 
        'job_id', 'job_name']

# come back to later: (lists) 'major_group_ids', 'canonical_major_group_ids', 'major_ids', 'major_names', 
job_cols = [ 'duration', 'employer_industry_id', 'employer_industry_name',  'employer_logo_url', 
            'job_type_id', 'job_type_name', 
            'salary_type_id', 'salary_type_name', 'text_description', 
            'title', 'work_study', 'remote']
#             'location_cities', 'location_countries', 'location_names', 'location_states'] 
# Removed: id, 'division_id', 'division_name', 'employer_group_ids',
# location_cols = ['location_names_1', 'location_names_2', 'location_names_3', 'location_names_4', 'location_names_5']

# removed: 'messageable_job_user_profile', 'scheduleable_job_user_profile', 'job_apply_setting',
# 'required_job_document_types', 'student_qualifications', 'canonical_major_group_ids', 'job_role_group_ids'

# posting_quality_cols = ['time_decayed_popularity_score', 'is_low_quality']
# student_screen_cols = ['type', 'cumulative_gpa_required', # 'cumulative_gpa', 
#                       'accepts_opt_cpt_candidates', 'willing_to_sponsor_candidate', 
#                       'graduation_date_minimum', 'graduation_date_maximum', 'work_auth_required']
#                       'school_year_or_graduation_date_required', 'us_authorization_optional', 
#                       'work_authorization_requirements', 'major_ids', 'major_group_ids', 'major_group_ids_count', 
#                       'college_ids', 'school_year_ids', 'institution_label_ids', 'custom_job_interest_option_ids']


location_cols = []
for i in range(1, 6): 
    for x in ["location_cities", "location_states", "location_countries"]: 
        location_cols.append(f"{x}_{i}")

all_cols = cols + job_cols + location_cols
jobs_df_dict = {colname: [] for colname in all_cols}

for line in data['postings']:   
    for col in cols: 
        try:                 
            jobs_df_dict[col].append(line[col])
        except: 
            jobs_df_dict[col].append(None)
            
    for col in job_cols:
        try: 
            jobs_df_dict[col].append(line['job'][col])
        except: 
            jobs_df_dict[col].append(None) 
    
    for col in location_cols: 
        colname = col[0:-2]
        loc_count = int(col[-1]) - 1
        try: 
            jobs_df_dict[col].append(line['job'][colname][loc_count])
        except: 
            jobs_df_dict[col].append(None)
        loc_count += 1
        
    # for col in student_screen_cols: 
    #     try: 
    #         jobs_df_dict[col].append(line['job']['student_screen'][col])
    #     except: 
    #         jobs_df_dict[col].append(None)

jobs_df = pd.DataFrame.from_dict(jobs_df_dict)

In [11]:
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37803 entries, 0 to 37802
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      37803 non-null  int64  
 1   apply_start             37790 non-null  object 
 2   created_at              37803 non-null  object 
 3   updated_at              37803 non-null  object 
 4   employer_logo           37803 non-null  object 
 5   employer_name           37803 non-null  object 
 6   employer_id             37803 non-null  int64  
 7   employment_type_name    37803 non-null  object 
 8   expiration_date         37803 non-null  object 
 9   job_id                  37803 non-null  int64  
 10  job_name                37803 non-null  object 
 11  duration                37788 non-null  object 
 12  employer_industry_id    37795 non-null  float64
 13  employer_industry_name  37795 non-null  object 
 14  employer_logo_url       37803 non-null

## Parse dates/times

In [14]:
datetime_cols = ["apply_start", "created_at", "updated_at", "expiration_date"]
for col in datetime_cols: 
    jobs_df[col] = jobs_df[col].apply(lambda x: pd.to_datetime(x))

## Manual fixes countries

In [15]:
country_cols = ["location_countries_1", "location_countries_2", "location_countries_3", 
                "location_countries_4", "location_countries_5"]
for c in country_cols: 
    jobs_df[col] = jobs_df[col].apply(lambda x: "United States" if x == "United State of America" else x)

In [16]:
jobs_df["location_countries_1"].value_counts()[0:20]

United States                 34813
United States of America        919
People's Republic of China      368
United Kingdom                  100
Canada                           89
Hong Kong                        60
Japan                            51
South Africa                     41
Taiwan                           41
Singapore                        32
Germany                          21
South Korea                      20
Puerto Rico                      20
Australia                        19
Malaysia                         18
India                            14
Malawi                           13
Ireland                          12
France                           12
Spain                            11
Name: location_countries_1, dtype: int64

# Matching to TheOrg Employer IDs

In [17]:
len(jobs_df["employer_name"].unique())

6190

In [33]:
jobs_df.to_csv("handshake_jobs.csv")

In [ ]:
# Close file
f.close()